In [1]:
import numpy as np
from gym import Env
from gym.spaces import Box, Discrete
import random

In [2]:
class ChargingEnv(Env):
    def __init__(self, max_charging_rate=22, mu=30, sigma=5):
        self.max_charging_rate = max_charging_rate
        self.battery_capacity = 100
        self.mu = mu
        self.sigma = sigma
        self.action_space = Discrete(4)
        self.observation_space = Box(low=np.array([0]), high=np.array([self.battery_capacity]))
        self.state = 10 + random.randint(-5,5) # starting battery load
        self.charging_length = 120

    def step(self, action): 
        charging_rate = action * (self.max_charging_rate/3) # assuming highest rate is 22 per 15min
        self.state += charging_rate 
        self.state = min(self.state, self.battery_capacity)
        self.charging_length -= 15

        # Checking if charging is done
        if self.charging_length <= 0: 
            done = True
        else:
            done = False

        # Calculate charging cost
        charging_cost = 0.25 * np.exp(charging_rate)  # as per the given formula, assuming alpha t is 0.25 for the 15min interval
        
        # Calculating the reward
        if done:
            energy_demand = np.random.normal(self.mu, self.sigma)
            if self.state < energy_demand:
                reward = -10000  # Penalty for running out of energy
            else:
                # Reward is proportional to the energy saved
                reward = -charging_cost
        else:
            reward = -charging_cost
        
        
        
        # Setting the placeholder for info
        info = {}
        
        # Returning the step information
        return self.state, reward, done, info
    
    def reset(self):
        self.state = 0
        self.charging_length = 120 
        return self.state

In [3]:
env = ChargingEnv()

/Users/julialauten/git/aaa/.venv/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
episodes = 20 #20 shower episodes
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-2343119.039616757
Episode:2 Score:-1793628365.4543197
Episode:3 Score:-1794799159.2366972
Episode:4 Score:-897985359.8782529
Episode:5 Score:-896815713.2020215
Episode:6 Score:-897985742.2469683
Episode:7 Score:-2689270988.6616707
Episode:8 Score:-897400154.1714218
Episode:9 Score:-1793628365.4543197
Episode:10 Score:-896815330.8333061
Episode:11 Score:-1793043924.4849198
Episode:12 Score:-1793627983.0856042
Episode:13 Score:-1758295.7015011082
Episode:14 Score:-2689270606.2929554
Episode:15 Score:-1793043159.7474887
Episode:16 Score:-2689270988.661671
Episode:17 Score:-3586084405.6513996
Episode:18 Score:-1794213953.5298662
Episode:19 Score:-3513912.8219943037
Episode:20 Score:-897400154.1714218


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [6]:
states = env.observation_space.shape
actions = env.action_space.n

In [7]:
model = Sequential()    
model.add(Dense(24, activation='relu', input_shape=states))
model.add(Dense(24, activation='relu'))
model.add(Dense(actions, activation='linear'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 4)                 100       
                                                                 
Total params: 748 (2.92 KB)
Trainable params: 748 (2.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [10]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [11]:
dqn = build_agent(model, actions)
del model
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=60000, visualize=False, verbose=1)

AttributeError: 'Sequential' object has no attribute '_compile_time_distribution_strategy'

In [10]:
# Defining and training of Deep Q-Network Agent
policy = BoltzmannQPolicy()
memory = SequentialMemory(limit=10000*2, window_length=1)
dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10000*1, target_model_update=1e-2)
dqn.compile(Adam(learning_rate=0.1), metrics=['mae'])
dqn.fit(env, nb_steps=10000*10, visualize=False, verbose=1)

NameError: name 'rl' is not defined

In [ ]:
# Testing the agent on the simple Environment
results = dqn.test(printEnv, nb_episodes=2, visualize=False)
print(np.mean(results.history['episode_reward']))